In [ ]:
import openai
import pandas as pd
import re

# Set your OpenAI API key
openai.api_key = "tG0doyJx_lA3EhbSOUkENn3V5R1qn4Q06CKvYL-oA"  # Replace with your actual key


def rate_call_sentiment_df(transcript, model="gpt-4"):
    prompt = (
        "You are a sentiment analysis expert.\n"
        "Given a customer support call transcript, rate the overall sentiment of the customer's experience on a scale from 1 to 10, where:\n"
        "- 1 = Extremely negative experience\n"
        "- 5 = Neutral or mixed experience\n"
        "- 10 = Extremely positive experience\n\n"
        "Consider the customer’s tone, language, satisfaction, and resolution of the issue.\n"
        "After giving the score, briefly explain in 2–3 bullet points why you chose that score.\n\n"
        f"Here is the transcript:\n{transcript}"
    )

    response = openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful assistant for sentiment scoring."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2
    )

    output = response.choices[0].message.content.strip()

    # Extract score
    score = None
    for match in re.findall(r'\b\d+\b', output):
        num = int(match)
        if 1 <= num <= 10:
            score = num
            break

    # Clean explanation
    explanation = output
    if score is not None:
        explanation = re.sub(rf".*?\b{score}\b[.:]?\s*", "", output, count=1, flags=re.IGNORECASE | re.DOTALL).strip()

    return score, explanation

In [3]:
rate_call_sentiment_df('I wanted to share some thoughts on my recent order. While I did eventually receive the package, it arrived a few days later than expected. I appreciate that the product quality is good and matches the description on your website, but the delay and lack of proactive updates made the experience frustrating. I had to follow up twice before getting a response. I hope you can improve communication in the future. I’m not entirely unhappy, but the experience wasn’t as smooth as it could have been.')

(4,
 '- The customer expressed frustration due to the delay in delivery and lack of proactive updates. They had to follow up twice before getting a response, which indicates a negative experience.\n- The customer appreciated the product quality and mentioned that it matches the description on the website, which is a positive aspect.\n- The customer is not entirely unhappy but suggests room for improvement, indicating a mixed but leaning towards negative experience.')